**Takeaway** <br>
1. It is much easier to mapping Llama embeddings (dim = 4096) to KG embeddings (dim ~ 3), given that Llama embeddings have higher dimensions.

# Set up

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
# data import
import os
import pickle
path_llama_emb = "/Users/yuanchenwei/Desktop/Capstone Project/triplets_llama_embeddings.pkl"
path_pykeen_emb = "/Users/yuanchenwei/Desktop/Capstone Project/pykeen_embed.pkl"

with open(path_llama_emb, 'rb') as f:
    llama_emb = pickle.load(f) # embedding from llama -> x

with open(path_pykeen_emb, 'rb') as f:
    pykeen_emb = pickle.load(f) # embedding from pykeen -> y

In [ ]:
x = []

In [6]:
llama_emb["Women"].mean(dim = 1)

tensor([[-0.9619, -0.6189,  0.7819,  ...,  1.1022,  0.4357,  0.8974]])

In [3]:
# set up x and y, step 1
x_list_pad = []
x_list_avg = []
y_list = []
for entity in llama_emb.keys():
    # x_list_pad.append(llama_emb[entity].squeeze(dim = 0))
    x_list_avg.append(llama_emb[entity].mean(dim = 1))

: 

In [ ]:
# set up x and y, step 1
x_list_pad = []
x_list_avg = []
y_list = []
for entity in llama_emb.keys():
    x_list_pad.append(llama_emb[entity].squeeze(dim = 0)) # change [1, 4, 4096] to [4, 4096]
    x_list_avg.append(llama_emb[entity].mean(dim = 1)) # # change [1, 4, 4096] to [1, 4096]
    y_list.append(pykeen_emb[entity].unsqueeze(dim = 0)) # change [32] to [1, 32]


In [ ]:
data = pd.DataFrame({"x_pad": x_list_pad, "x_avg": x_list_avg, "y": y_list})
data_train, data_test = train_test_split(data, test_size=0.1)

In [ ]:
# def collate_pad(batch):
    

In [ ]:
# set up x and y, step 2
x_pad_train = nn.utils.rnn.pad_sequence(data_train["x_pad"], batch_first=True) # shape: [156, 15, 4096]
x_avg_train = torch.cat(data_train["x_avg"].to_list()) # shape[156, 4096]
y_train = torch.cat(data_train["y"].to_list()) # shape: [156, 32]
print(f"x_pad: {x_pad_train.shape}")
print(f"x_avg: {x_avg_train.shape}")
print(f"y: {y_train.shape}")

In [ ]:
# set up x and y, step 2
x_pad_test = nn.utils.rnn.pad_sequence(data_test["x_pad"], batch_first=True) # shape: [156, 15, 4096]
x_avg_test = torch.cat(data_test["x_avg"].to_list()) # shape[156, 4096]
y_test = torch.cat(data_test["y"].to_list()) # shape: [156, 32]
print(f"x_pad: {x_pad_test.shape}")
print(f"x_avg: {x_avg_test.shape}")
print(f"y: {y_test.shape}")

In [ ]:
# set up dataloader
BATCH_SIZE = 16
data_x_pad_train = DataLoader(x_pad_train, batch_size = BATCH_SIZE, shuffle = False) # recall that BATCH_SIZE = 16
data_x_avg_train = DataLoader(x_avg_train, batch_size = BATCH_SIZE, shuffle = False)
data_y_train = DataLoader(y_train, batch_size = BATCH_SIZE, shuffle = False)
data_x_pad_test = DataLoader(x_pad_test, batch_size = BATCH_SIZE, shuffle = False)
data_x_avg_test = DataLoader(x_avg_test, batch_size = BATCH_SIZE, shuffle = False)
data_y_test = DataLoader(y_test, batch_size = BATCH_SIZE, shuffle = False)
# train_data_avg = [(xi, yi) for xi, yi in zip(x_avg, y)]
# trainloader_avg = DataLoader(train_data_avg, batch_size = BATCH_SIZE, shuffle = True) # recall that BATCH_SIZE = 16

In [ ]:
for x in data_x_pad_test:
    print(x)
    break

In [ ]:
class Mapping(nn.Module):
  def __init__(self, input_size, hidden_size_lstm, hidden_size_lin, output_size):
    super(Mapping, self).__init__()
    # self.hidden_size_lstm = hidden_size_lstm
    self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size_lstm, batch_first = True)
    #self.fc1 = nn.Linear(input_size, hidden_size)
    self.fc1 = nn.Linear(hidden_size_lstm, hidden_size_lin)
    self.fc2 = nn.Linear(hidden_size_lin, output_size)
    self.relu = nn.ReLU()

  def forward(self, x, hidden_cell = None):
    #print(f"x: {x.shape}")
    # x -> [Batch_size, length, D]
    output, (hn, _) = self.lstm(x, hidden_cell)
    #print(f"hn: {hn.shape}")
    # hn -> [Batch_size, length, hidden_size]
    #hn = nn.ReLU()(hn)
    
    x = self.fc1(hn)
    x = self.relu(x)
    x = self.fc2(x)
    #print(f"output: {output.shape}")
    return x
    

In [ ]:
# recall that
INPUT_SIZE = 4096
HIDDEN_SIZE_LSTM = 512
HIDDEN_SIZE_LIN = 128
OUTPUT_SIZE = 32
model = Mapping(INPUT_SIZE, HIDDEN_SIZE_LSTM, HIDDEN_SIZE_LIN, OUTPUT_SIZE)
#model = nn.Linear(INPUT_SIZE, OUTPUT_SIZE)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

#def loss_func(feat1, feat2):
#    return - nn.functional.cosine_similarity(feat1, feat2).mean()

#loss_fn = loss_func

In [ ]:
# Training
EPOCHS = 100

for epoch in range(EPOCHS):

  for x_batch, y_batch in zip(data_x_pad_train, data_y_train):

    optimizer.zero_grad()

    pred = model(x_batch)
    
    loss = loss_fn(pred, y_batch)

    # Get teh gradients of l with respect to theta
    loss.backward()

    # Update theta
    optimizer.step()

  if (epoch + 1) % 10 == 0:   
    print('Epoch:{} Loss: {:.4f}'.format(epoch + 1, loss))

In [ ]:
# Evaluation
cos = nn.CosineSimilarity(dim=1, eps=1e-6)

with torch.no_grad():
  pred = model(x).squeeze(dim = 0) # shape: [156, 32]
  similarity = cos(pred, y)
  print(similarity)



In [ ]:
i = 1
print(y[i])
print(pred[i])